The goal of this notebook is to check the automated occlusion annotations produced by the Unity VizTest tool built by Scott MacDonald against the ones human annotators did by hand. See this Quip document for details: https://fb.quip.com/GuSYAz9JosOv

In [39]:
import numpy as np
import pandas as pd
import visual_search_load
import re
import visual_search_objects 

### Preprocess annotation results for one scene.

In [40]:
## Set directories. 
# Main data directory. 
direc = '/Users/angelaradulescu/Dropbox/NYU/Research/TaskVR/VisualSearchRawData/'
# Hand annotations directory. 
hand_occlusion_direc = '/Users/angelaradulescu/Dropbox/NYU/Research/TaskVR/VisualSearchGazeData/occlusion_hand_annotated_scenes/'
# VizTest automated annotations directory. 
auto_occlusion_direc = '/Users/angelaradulescu/Dropbox/NYU/Research/TaskVR/Unity_projects/Occlusion_tagging/VizTestOutput/'
data_direc, participant_list = visual_search_load.get_direc_participants('Expt2', direc)
# Locations directory. 
locations_direc = '/Users/angelaradulescu/Dropbox/NYU/Research/TaskVR/VisualSearchGazeData/location/'

## Load locations.
locations = pd.read_csv(locations_direc + 'location_all_objects.csv')

## Grab all scenes.
indexes = np.unique(locations['scene'].values, return_index=True)[1]
all_scenes = [locations['scene'].values[index] for index in sorted(indexes)]

In [46]:
## Select scene. 
scene = 'Bathroom Location 3 Trial 7'
# scene = 'Guest Bedroom Location 5 Trial 10'

In [47]:
## Get manual annotation. 
objects = visual_search_load.get_all_objects(data_direc, scene)
locations_str = visual_search_load.get_object_locations(data_direc, scene)
locations_arr = visual_search_objects.parse_locations(locations_str)
hand_occlusion_labels = pd.read_csv(hand_occlusion_direc + scene + '.csv') 
hand_occlusion_labels = hand_occlusion_labels.drop(columns='Unnamed: 0')
hand_occlusion_labels.insert(0, 'object', value=objects)
hand_occlusion_labels.insert(2, 'location', value=locations_arr)

hand_occlusion_labels[hand_occlusion_labels['object'] == 'Soccer Ball']
hand_occlusion_labels.head(5)

,object,occluded,location
0,Book 1,0.0,"[-4.763133, 0.761089, 6.695295]"
1,Paper Box,0.0,"[-5.058631, 0.133333, 8.123557]"
2,Vase 1,1.0,"[-1.987741, 0.193434, 8.58199]"
3,Block 2,1.0,"[-2.070976, 0.174345, 8.427336]"
4,Tomato,0.0,"[-4.600757, 0.798804, 6.620316]"


In [48]:
## Get auto annotation. 
auto_occlusion_labels = visual_search_objects.parse_auto_occlusion_tags(auto_occlusion_direc + 'results_' + scene)
auto_occlusion_labels.head(5)

,object,occluded,location
0,Pizza Box,0,"[-3.36, 0.028, 7.816]"
1,Pen,0,"[-1.763, 0.004, 6.907]"
2,Crayon 3,1,"[-3.361, 0.005, 8.237]"
3,Rubber Duck,0,"[-3.134, 0.065, 7.161]"
4,Block 3,0,"[-4.391, 0.029, 7.618]"


In [49]:
## Combine auto and hand annotations.
combined_labels = visual_search_objects.match_occlusion_labels(auto_occlusion_labels, hand_occlusion_labels)
combined_labels.head(10)

,object,occluded,location,nearest_auto_annot_obj,nearest_occluded,nearest_location
0,Book 1,0.0,"[-4.763133, 0.761089, 6.695295]",Book 1,0,"[-4.763, 0.761, 6.695]"
1,Paper Box,0.0,"[-5.058631, 0.133333, 8.123557]",Paper Box,0,"[-5.059, 0.133, 8.124]"
2,Vase 1,1.0,"[-1.987741, 0.193434, 8.58199]",Vase 1,0,"[-1.988, 0.193, 8.582]"
3,Block 2,1.0,"[-2.070976, 0.174345, 8.427336]",Block 2,1,"[-2.071, 0.174, 8.427]"
4,Tomato,0.0,"[-4.600757, 0.798804, 6.620316]",Tomato_LOD2,0,"[-4.601, 0.799, 6.62]"
5,Scissors,0.0,"[-5.487945, 0.124983, 8.110414]",Scissors,0,"[-5.488, 0.125, 8.11]"
6,Dodecahedron,0.0,"[-5.594612, 0.278637, 8.192053]",Dodecahedron,0,"[-5.595, 0.279, 8.192]"
7,Crayon 6,1.0,"[-2.216746, 0.182995, 8.570617]",Crayon 6,1,"[-2.217, 0.183, 8.571]"
8,Laptop,0.0,"[-3.683148, 0.88329, 8.172914]",laptop,0,"[-3.683, 0.883, 8.173]"
9,Picture 2,1.0,"[-4.971868, 0.17841, 6.742446]",Picture 2,1,"[-4.972, 0.178, 6.742]"


In [50]:
## Compute agreement. 
# Overall.
a = np.round(np.mean(combined_labels['occluded'].values == combined_labels['nearest_occluded'].values),3)
print('overall agreement between human and tool: ' + str(a))
# On occlusions.
idx = np.where(combined_labels['occluded'].values == 1)[0]
a = np.round(np.sum(combined_labels.loc[idx]['nearest_occluded'].values)/len(idx),3)
so = np.sum(combined_labels.loc[idx]['nearest_occluded'].values)
print('number of occlusions: ' + str(len(idx)))
print('agreement on occlusions: ' + str(a) + ', ' + str(so) + '/' + str(len(idx)))

overall agreement between human and tool: 0.932
number of occlusions: 11
agreement on occlusions: 0.818, 9/11


### Examine fail cases. 

In [17]:
x = np.where(hand_occlusion_labels['occluded'].values != hand_occlusion_labels['nearest_occluded'].values)[0]
hand_occlusion_labels.loc[x]

KeyError: 'nearest_occluded'

In [13]:
hand_occlusion_labels[hand_occlusion_labels['nearest_auto_annot_obj'] == 'Book 1']

KeyError: 'nearest_auto_annot_obj'

In [14]:
print(auto_occlusion_labels[auto_occlusion_labels['object'] == 'Book 1'])
auto_occlusion_labels[(auto_occlusion_labels['unique'] > 20) & (auto_occlusion_labels['unique'] < 30)]

Empty DataFrame
Columns: [object, occluded, location, unique]
Index: []


,object,occluded,location,unique
22,Clipboard,1,"[0.598, 0.785, 6.642]",21
23,Donut 2,1,"[1.588, 0.62, 8.207]",22
24,Goods-Tin_LOD2,1,"[1.664, 0.408, 10.735]",23
25,Goods-Tin,1,"[1.664, 0.408, 10.735]",23
26,Goods-Tin,1,"[1.664, 0.408, 10.735]",23
27,Magazine 1,1,"[1.364, 0.235, 6.645]",24
28,ITM3D_CoconutHalf2_LOD2,1,"[2.658, 0.494, 10.483]",25
29,ITM3D_CoconutHalf2,1,"[2.658, 0.494, 10.483]",25
30,ITM3D_CoconutHalf2,1,"[2.658, 0.494, 10.483]",25
31,Scissors,1,"[1.446, 0.606, 9.114]",26


In [15]:
target_loc = np.array([5.539, 0.733, 0.383])
np.where(np.sum(np.vstack(auto_occlusion_labels['location'].values) == target_loc, axis=1))[0]

array([], dtype=int64)

### Introduce auto occlusion tags in the location files. 

In [71]:
def generate_occlusion_labels(data_direc, auto_occlusion_direc, locations):
    
    ## Make a copy of the dataframe.
    locations_tagged = locations.copy()
    locations_tagged['occluded'] = np.nan
    
    ## Grab all scenes.
    indexes = np.unique(locations['scene'].values, return_index=True)[1]
    all_scenes = [locations['scene'].values[index] for index in sorted(indexes)]
    
    df_list = []
    
    ## Loop through scenes.
    for scene in all_scenes:
        
        print(scene)
        
        ## Grab auto annotations for this scene.
        auto_occlusion_labels = visual_search_objects.parse_auto_occlusion_tags(auto_occlusion_direc + 'results_' + scene)
        
        if (scene == 'Guest Bedroom Location 5 Trial 10'):
            print('problem scene')
            hand_occlusion_labels = pd.read_csv(hand_occlusion_direc + scene + '.csv')
        
        ## Subset locations for this scene.
        locations_scene = locations[locations['scene'] == scene].reset_index(drop=True)
        locations_str = visual_search_load.get_object_locations(data_direc, scene)
        locations_arr = visual_search_objects.parse_locations(locations_str)
        locations_scene['location'] = locations_arr
        
        ## Combine manual and annotated 
        combined = visual_search_objects.match_occlusion_labels(auto_occlusion_labels, locations_scene)
        print('target occluded? ' + str(combined['nearest_occluded'].values[0]))
        print('percentage occluded: ' + str(np.sum(combined['nearest_occluded'].values)/len(locations_scene)))
        locations_scene['occluded'] = combined['nearest_occluded'].values
        
        if (scene == 'Guest Bedroom Location 5 Trial 10'):
            print('replacing automated labels with hand labels')
            locations_scene['occluded'] = hand_occlusion_labels['occluded'].values
           
        df_list.append(locations_scene)
    
    locations_tagged = pd.concat(df_list)    
         
    return locations_tagged

locations_tagged = generate_occlusion_labels(data_direc, auto_occlusion_direc, locations)
locations_tagged.to_csv(locations_direc+'location_all_objects_tagged.csv', index=False)

Kitchen Location 1 Trial 9
target occluded? 0
percentage occluded: 0.0851063829787234
Kitchen Location 2 Trial 3
target occluded? 0
percentage occluded: 0.15789473684210525
Kitchen Location 2 Trial 4
target occluded? 0
percentage occluded: 0.10869565217391304
Living Room Location 4 Trial 5
target occluded? 0
percentage occluded: 0.32710280373831774
Studio Location 3 Trial 3
target occluded? 0
percentage occluded: 0.078125
Studio Location 5 Trial 3
target occluded? 0
percentage occluded: 0.1791044776119403
Bedroom Location 5 Trial 5
target occluded? 0
percentage occluded: 0.2236842105263158
Guest Bedroom Location 3 Trial 5
target occluded? 0
percentage occluded: 0.0967741935483871
Living Room Location 5 Trial 10
target occluded? 0
percentage occluded: 0.32142857142857145
Bedroom Location 1 Trial 8
target occluded? 0
percentage occluded: 0.0821917808219178
Bedroom Location 4 Trial 2
target occluded? 0
percentage occluded: 0.13157894736842105
Studio Location 2 Trial 1
target occluded? 0
p

target occluded? 0
percentage occluded: 0.2920353982300885
Studio Location 4 Trial 10
target occluded? 0
percentage occluded: 0.09230769230769231
Studio Location 2 Trial 3
target occluded? 0
percentage occluded: 0.14925373134328357
Bathroom Location 1 Trial 7
target occluded? 0
percentage occluded: 0.1864406779661017
Studio Location 2 Trial 2
target occluded? 0
percentage occluded: 0.203125
Studio Location 5 Trial 8
target occluded? 0
percentage occluded: 0.171875
Kitchen Location 4 Trial 8
target occluded? 0
percentage occluded: 0.18947368421052632
Living Room Location 2 Trial 8
target occluded? 0
percentage occluded: 0.3211009174311927
Bathroom Location 3 Trial 8
target occluded? 0
percentage occluded: 0.1896551724137931
Guest Bedroom Location 2 Trial 3
target occluded? 0
percentage occluded: 0.17708333333333334
Living Room Location 3 Trial 6
target occluded? 0
percentage occluded: 0.16964285714285715
Bathroom Location 1 Trial 2
target occluded? 0
percentage occluded: 0.2131147540983

target occluded? 0
percentage occluded: 0.19148936170212766
Kitchen Location 3 Trial 2
target occluded? 0
percentage occluded: 0.21978021978021978
Bathroom Location 4 Trial 1
target occluded? 0
percentage occluded: 0.2807017543859649
Bathroom Location 4 Trial 8
target occluded? 0
percentage occluded: 0.25862068965517243
Studio Location 5 Trial 10
target occluded? 0
percentage occluded: 0.10144927536231885
Bedroom Location 2 Trial 9
target occluded? 0
percentage occluded: 0.19444444444444445
Guest Bedroom Location 3 Trial 3
target occluded? 0
percentage occluded: 0.042105263157894736
Studio Location 4 Trial 2
target occluded? 0
percentage occluded: 0.11764705882352941
Guest Bedroom Location 3 Trial 4
target occluded? 0
percentage occluded: 0.06521739130434782
Guest Bedroom Location 2 Trial 4
target occluded? 0
percentage occluded: 0.2
Living Room Location 3 Trial 5
target occluded? 1
percentage occluded: 0.2
Studio Location 4 Trial 5
target occluded? 0
percentage occluded: 0.17647058823

target occluded? 0
percentage occluded: 0.16666666666666666
Bedroom Location 3 Trial 2
target occluded? 0
percentage occluded: 0.0547945205479452
Studio Location 1 Trial 3
target occluded? 0
percentage occluded: 0.046153846153846156
Kitchen Location 4 Trial 2
target occluded? 0
percentage occluded: 0.22340425531914893
Guest Bedroom Location 1 Trial 10
target occluded? 0
percentage occluded: 0.22916666666666666
Guest Bedroom Location 1 Trial 7
target occluded? 0
percentage occluded: 0.22340425531914893
Kitchen Location 3 Trial 8
target occluded? 0
percentage occluded: 0.24468085106382978
Kitchen Location 4 Trial 3
target occluded? 0
percentage occluded: 0.15789473684210525
Living Room Location 2 Trial 10
target occluded? 0
percentage occluded: 0.26126126126126126
Studio Location 4 Trial 8
target occluded? 0
percentage occluded: 0.19696969696969696
Studio Location 2 Trial 4
target occluded? 0
percentage occluded: 0.13432835820895522
Bathroom Location 1 Trial 10
target occluded? 0
percent

In [72]:
locations_tagged[locations_tagged['scene'] == 'Guest Bedroom Location 5 Trial 10']

,scene,object,object_idx,x,y,z,r_polar,theta_polar,x_pos_pixel_360,y_pos_pixel_360,location,occluded
0,Guest Bedroom Location 5 Trial 10,Juice Box,0,1.815701,-0.440673,2.524845,0.623452,-0.140762,4909.0,2232.0,"[-0.005111, 1.259327, 10.3356]",0
1,Guest Bedroom Location 5 Trial 10,Soda Can,1,-0.257768,-1.650431,0.573016,-0.422725,-1.207035,3545.0,3622.0,"[2.310593, 0.049569, 8.678365]",0
2,Guest Bedroom Location 5 Trial 10,Crayon 1,2,-2.096002,-1.465555,2.204020,-0.760283,-0.449020,3105.0,2633.0,"[1.065534, 0.234445, 6.559614]",0
3,Guest Bedroom Location 5 Trial 10,Notebook,3,-0.410877,-0.893913,-0.332960,-2.251824,-1.036567,1160.0,3399.0,"[3.229079, 0.806087, 8.703205]",0
4,Guest Bedroom Location 5 Trial 10,Eraser,4,-0.819050,-0.938475,-0.266358,-1.885213,-0.828269,1638.0,3128.0,"[3.242605, 0.761525, 8.289856]",0
...,...,...,...,...,...,...,...,...,...,...,...,...
90,Guest Bedroom Location 5 Trial 10,Takeout Box,90,1.274587,-1.641329,2.550072,0.463507,-0.522383,4700.0,2729.0,"[0.074698, 0.058671, 9.799809]",1
91,Guest Bedroom Location 5 Trial 10,Block 10,91,-0.837748,-1.037320,1.250800,-0.590148,-0.603340,3327.0,2835.0,"[1.757654, 0.66268, 7.978346]",0
92,Guest Bedroom Location 5 Trial 10,Tea Box,92,-1.527747,-1.641756,0.277691,-1.390994,-0.813244,2282.0,3108.0,"[2.845752, 0.058244, 7.489387]",0
93,Guest Bedroom Location 5 Trial 10,Block 4,93,1.449727,-1.289594,-0.397374,1.838328,-0.709061,6493.0,2972.0,"[2.932751, 0.410406, 10.54119]",0
